# VIDEO2AUDIO

In [1]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip
import subprocess
import os

def video2audio(name):
    file = 'video/'+name+'.mp4'
    p = subprocess.call('ffmpeg -i '+file+' -q:a 0 -map a video/'+name+'.wav -y', shell = True)

# AUDIO2TEXT

In [2]:
import speech_recognition as sr

def audio2text(name):
    english_text = ''
    r = sr.Recognizer()
    harvard = sr.AudioFile('video/'+name+'.wav')
    shift  = 35
    myduration = shift
    myoffset = 0
    try:
        while 1:
            with harvard as source:
                r.adjust_for_ambient_noise(source)
                audio = r.record(source, duration = myduration, offset = myoffset)
                myoffset += shift
                while 1:
                    qq = r.recognize_google(audio, language="en-US")
                    try:
                        if len(qq) != '':
                            break
                        else:
                            qq=''
                    except Exception as e: print(e) 
                english_text += qq
                english_text += ' '
    except Exception as e:
        print(e)
        
    return english_text

# TAGS

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline
import re
import time
import requests
import subprocess
import os
from moviepy.editor import VideoFileClip
from google_images_search import GoogleImagesSearch
from io import BytesIO
from PIL import Image
import numpy as np
import pandas as pd
from io import BytesIO
from PIL import Image, UnidentifiedImageError
import numpy as np
import pandas as pd
from google_trans_new import google_translator
from rutermextract import TermExtractor
from spacy import displacy
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import nltk
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import math
import subprocess
import spacy
import en_core_web_lg
import jellyfish
import datetime
import os
from collections import Counter
from collections import Counter
import subprocess
import os
from moviepy.editor import VideoFileClip
from textblob import TextBlob
from nltk.stem import PorterStemmer, WordNetLemmatizer
import jellyfish

lemmatizer = WordNetLemmatizer()
coronavirus = ['coronavirus', 'corona', 'virus', 'pandemic', 'covid19', 'moderna']
dictionary=[]
dictionary.append(coronavirus)

def init_corpus():

    corpus_book = ['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt',
    'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt',
    'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt',
    'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt',
    'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt',
    'shakespeare-macbeth.txt', 'whitman-leaves.txt']
    ttext = ''

    for j in corpus_book:
        emma = nltk.corpus.gutenberg.words(j)

        for i in emma:
            ttext +=i
            ttext +=' '
    database = []
    mass = ''
    counter = 0
    for jj in ttext:
        if jj == '.':
            if counter == 50:
                database.append(mass)
                counter = 0
                mass = ''
            else:
                counter += 1
        mass += jj
    return database

database = init_corpus()
nlp = en_core_web_lg.load()
translator = google_translator()
lemmatizer = WordNetLemmatizer()

def compare_stemmer_and_lemmatizer(stemmer, lemmatizer, word, pos):
    return lemmatizer.lemmatize(word, pos)

def compute_tfidf(corpus):
    def compute_tf(english_text):
        tf_text = Counter(english_text)
        for i in tf_text:
            tf_text[i] = tf_text[i]/float(len(english_text))
        return tf_text
    def compute_idf(word, corpus):
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))
    documents_list = []
    for english_text in corpus:
        tf_idf_dictionary = {}
        computed_tf = compute_tf(english_text)
    for word in computed_tf:
        tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

def freq(str):
    freq_dict = {}
    str_list = str.split()
    unique_words = set(str_list)
    for words in unique_words :
        freq_dict.update({words: str_list.count(words)})
    freq_dict = list(freq_dict.items())
    freq_dict.sort(key=lambda i: i[1])
    freq_dict.reverse()
    return freq_dict

def spacy_tags(english_text):
    doc = nlp(english_text)
    delimiter2 = 0
    while True:
        HighPrior = []
        LowPrior = []
        delimiter2 += 1
        try:
            for ent in doc.ents:
                if ent.label_ !='DATE' and ent.label_ !='NORP' and ent.label_ !='ORDINAL' and ent.label_ !='CARDINAL' and ent.label_ !='TIME' and ent.label_ !='QUANTITY':
                    HighPrior.append(lemmatizer.lemmatize(ent.text.lower()))
                else:
                    LowPrior.append(lemmatizer.lemmatize(ent.text.lower()))
        except Exception as e: print(e)
        if delimiter2 == 10 or len(HighPrior) != 0:
            break

    tags1type_spacy = list(set(HighPrior))
    tags1type_spacy = [lemmatizer.lemmatize(j) for j in tags1type_spacy]

    shift = 0
    for i in range(0,len(tags1type_spacy)):
        for j in range(0,len(tags1type_spacy)):
            try:
                if (str(tags1type_spacy[i+shift]).lower() in  str(tags1type_spacy[j]).lower() or str(tags1type_spacy[j]).lower() in str(tags1type_spacy[i+shift]).lower() or jellyfish.jaro_distance(str(tags1type_spacy[i+shift]).lower(), str(tags1type_spacy[j]).lower()) > 0.75 ) and jellyfish.jaro_distance(str(tags1type_spacy[i+shift]).lower(), str(tags1type_spacy[j]).lower()) != 1:
                    if len(tags1type_spacy[j]) > len(tags1type_spacy[i+shift]):
                        tags1type_spacy.remove(tags1type_spacy[i+shift])
                    else:
                        tags1type_spacy.remove(tags1type_spacy[j])
            except: pass
        shift+=1
    tags2type_spacy = list(set(LowPrior))
    tags2type_spacy = [lemmatizer.lemmatize(j) for j in tags2type_spacy]
    shift = 0
    for i in range(0,len(tags2type_spacy)):
        for j in range(0,len(tags2type_spacy)):
            try:
                if (str(tags2type_spacy[i+shift]).lower() in  str(tags2type_spacy[j]).lower() or str(tags2type_spacy[j]).lower() in str(tags2type_spacy[i+shift]).lower() or jellyfish.jaro_distance(str(tags2type_spacy[i+shift]).lower(), str(tags2type_spacy[j]).lower()) > 0.75 ) and jellyfish.jaro_distance(str(tags2type_spacy[i+shift]).lower(), str(tags2type_spacy[j]).lower()) != 1:
                    if len(tags2type_spacy[j]) > len(tags2type_spacy[i+shift]):
                        tags2type_spacy.remove(tags2type_spacy[i+shift])
                    else:
                        tags2type_spacy.remove(tags2type_spacy[j])
            except: pass
        shift+=1
    return tags1type_spacy, tags2type_spacy

def nltk_tags(english_text):

    lemmatizer = WordNetLemmatizer()
    result_count = []
    term_extractor = TermExtractor()
    translator = google_translator()
    term_extractor = TermExtractor()

    for term in term_extractor(english_text):
        if term.count >= 1:
            mm = term.normalized
            result_count.append(mm)
    return result_count

def tf_idm_tags(english_text, database):
    length = 0
    for l in database:
        length+=len(l)

    corpus = []
    stop_words = set(stopwords.words('english'))
    for text_block in database:
        data = []
        for jj in text_block.lower().replace('.','').replace(',','').replace('-','').split(' '):
            data.append(compare_stemmer_and_lemmatizer(PorterStemmer(), WordNetLemmatizer(), word = lemmatizer.lemmatize(jj), pos = wordnet.VERB))
        without_stop_words = [word for word in data if not word in stop_words]
        corpus.append(without_stop_words)
    length = 0
    for l in corpus:
        length+=len(l)

    vocabulary = list(compute_tfidf(corpus)[0].items())
    tf_idm = sorted(vocabulary, key=lambda tup: tup[1], reverse = True)
    ans = []
    [ans.append(j[0].lower().replace('"','').replace("'",'').replace('.','').replace(',','').replace('-','')) for j in tf_idm[0:5]]

    return (ans)

def get_tags(english_text):
    
    spacy1 = []
    spacy2 = []
    for i in english_text.split('. '):
        [spacy1.append(j) for j in spacy_tags(i)[0]]
        [spacy2.append(k) for k in spacy_tags(i)[1]]
    tags=[]
    blob = TextBlob(english_text)

    tags = [i for i in list(blob.noun_phrases)]
    tags = list(set(list(tags)))

    delete = []

    for j in tags:
        for k in tags:
            if jellyfish.jaro_distance(j, k) > 0.75 and jellyfish.jaro_distance(j, k)<0.99:
                if len(j)>len(k):
                    delete.append(j)
                else:
                    delete.append(k)
                if j in k and k!=j:
                    delete.append(j)
                if k in j and k!=j:
                    delete.append(k)
    for kk in delete:
        try:
            tags.remove(kk)
        except:pass
    [tags.append(i) for i in spacy1]
    tags = sorted(list(set(tags)))

    for i in tags:
        if len(i)<=1:
            tags.remove(i)
    stop_words = set(stopwords.words('english'))

    for i in list(stop_words):
        if i in tags:
            tags.remove(i)
            
    stop_words = set(stopwords.words('english'))

    for i in list(stop_words):
        if i in tags:
            tags.remove(i)

    tags2 = [j for j in tags]
    [tags2.append(i) for i in spacy2]
    tags2 = sorted(list(set(tags2)))
    d = {}
    for i in tags:
        if english_text.lower().find(i) != -1:
            d[english_text.lower().find(i)]=i
    tags = list(dict(sorted(d.items())).values())
    d = {}
    for i in tags2:
        if english_text.lower().find(i) != -1:
            d[english_text.lower().find(i)]=i
    tags2 = list(dict(sorted(d.items())).values())
                                                                   
    delete = []
    for j in tags:
        for k in tags:
            if j in k and k!=j:
                delete.append(j)
            elif k in j and k!=j:
                delete.append(k)
        for kk in delete:
            try:
                tags.remove(kk)
            except:pass
    delete = []
    for j in tags2:
        for k in tags2:
            if j in k and k!=j:
                delete.append(j)
            elif k in j and k!=j:
                delete.append(k)
    for kk in delete:
        try:
            tags2.remove(kk)
        except:pass
    return tags,tags2

# embeddings_similarity

In [4]:
import pandas as pd
import numpy as np
import string
import itertools
import spacy

def clean_text(sentence):
    clean_sentence = "".join(l for l in sentence if l not in string.punctuation)
    return clean_sentence

def cosine_similarity_calc(vec_1,vec_2):
    sim = np.dot(vec_1,vec_2)/(np.linalg.norm(vec_1)*np.linalg.norm(vec_2))
    return sim

def embeddings_similarity(sentences):
    sentence_pairs = list(itertools.combinations(sentences, 2))
    sentence_a = [pair[0] for pair in sentence_pairs]
    sentence_b = [pair[1] for pair in sentence_pairs]
    sentence_pairs_df = pd.DataFrame({'sentence_a':sentence_a, 'sentence_b':sentence_b})
    sentence_pairs_df = sentence_pairs_df.loc[pd.DataFrame(np.sort(sentence_pairs_df[['sentence_a', 'sentence_b']],1),index=sentence_pairs_df.index).drop_duplicates(keep='first').index]
    sentence_pairs_df = sentence_pairs_df[sentence_pairs_df['sentence_a'] != sentence_pairs_df['sentence_b']]
    embeddings = spacy.load('en_core_web_lg')
    sentence_pairs_df['similarity'] = sentence_pairs_df.apply(lambda row: cosine_similarity_calc(embeddings(clean_text(row['sentence_a'])).vector,
    embeddings(clean_text(row['sentence_b'])).vector), axis=1)
    return sentence_pairs_df

# Main DF creation

In [5]:
import pandas as pd

df = pd.read_excel('video\df.xlsx')
df['text'] = ''
df['tags'] = ''
df['metrics'] = ''

for i in range(0,len(df)):
    name = df.file[i].split('.')[0]
    video2audio(name)
    #df.text[i] = audio2text(name)
    #df.tags[i] = ', '.join(get_tags(df.text[i])[0])

# Find recommendations

In [87]:
name = 'animals/2.mp4'

def get_recommendation(name, df):
    ind = df.index[df['file'] == name].tolist()[0]
    tags_to_needed_article = df.tags[ind]
    
    for i in range(0,len(df)):
        sentences = [tags_to_needed_article, df.tags[i]]
        print(sentences)
        df.metrics[i] = embeddings_similarity(sentences).similarity[0]
        
    return df

get_recommendation(name, df1)

['europe, wild hamsters, grasslands tv, court desert, calories hamster cheeks, body weight, big cheeks, narrow bottlenecks', "strictly daphne, mobility service, bridget, lily anthony's, 's story 4jobs, trio show, disabilities weather human rk900, brasted, joy, languages touch, body rubs, hymn, 'm feeling, way chicken, puppy training, foster, bots, dual bike, way shape, 's differences"]


KeyboardInterrupt: 